In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Load the loss landscape data
data_path = 'results/loss_landscape/u_obs_sms_relative_loss_surface.npz'
data = np.load(data_path, allow_pickle=True)

# Extract the arrays
alpha_bg_values = data['alpha_bg_values']  # x-axis
alpha_sp_factors = data['alpha_sp_factors']  # y-axis
alpha_sp_values = data['alpha_sp_values']  # actual alpha_sp values
relative_loss = data['relative_loss']  # z-axis (loss values)

print(f"Data shape: {relative_loss.shape}")
print(f"Alpha_bg range: [{alpha_bg_values.min():.2f}, {alpha_bg_values.max():.2f}]")
print(f"Alpha_sp_factor range: [{alpha_sp_factors.min():.2f}, {alpha_sp_factors.max():.2f}]")
print(f"Relative loss range: [{np.nanmin(relative_loss):.6f}, {np.nanmax(relative_loss):.6f}]")
print(f"\nMinimum loss location:")
min_idx = np.unravel_index(np.nanargmin(relative_loss), relative_loss.shape)
print(f"  alpha_bg = {alpha_bg_values[min_idx[0]]:.2f}")
print(f"  alpha_sp_factor = {alpha_sp_factors[min_idx[1]]:.2f}")
print(f"  alpha_sp = {alpha_sp_values[min_idx]:.2f}")
print(f"  loss = {relative_loss[min_idx]:.6f}")

## 3D Surface Plot

Interactive 3D visualization of the loss landscape:
- **X-axis**: alpha_bg (background stiffness)
- **Y-axis**: alpha_sp_factor (special region stiffness factor)
- **Z-axis**: Relative loss
- **Color**: Loss magnitude (thermal colorscale)


## 2D Cross-Section: Varying alpha_sp_factor

Interactive 2D view with slider to explore cross-sections by fixing alpha_bg and varying alpha_sp_factor.


In [ ]:
# Create meshgrid for 3D surface plot
X, Y = np.meshgrid(alpha_bg_values, alpha_sp_factors, indexing='ij')
Z = relative_loss

# Calculate global z-axis range for consistent scaling
z_min = np.nanmin(relative_loss)
z_max = np.nanmax(relative_loss)
z_range = [z_min, z_max]

# Create 3D surface plot
fig_3d = go.Figure(data=[
    go.Surface(
        x=X,
        y=Y,
        z=Z,
        colorscale='thermal',
        colorbar=dict(
            title="Relative Loss",
            len=0.75
        ),
        contours=dict(
            z=dict(
                show=True,
                usecolormap=True,
                highlightcolor="limegreen",
                project=dict(z=True)
            )
        )
    )
])

# Add global minimum marker
fig_3d.add_trace(go.Scatter3d(
    x=[alpha_bg_values[min_idx[0]]],
    y=[alpha_sp_factors[min_idx[1]]],
    z=[relative_loss[min_idx] * 1.01],  # Slightly above surface
    mode='markers',
    marker=dict(
        size=10,
        color='lime',
        symbol='diamond',
        line=dict(width=2, color='black')
    ),
    name='Global Minimum',
    showlegend=True
))

# Update layout
fig_3d.update_layout(
    title=dict(
        text='Loss Landscape: 3D Surface View',
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    scene=dict(
        xaxis_title='alpha_bg (Background Stiffness)',
        yaxis_title='alpha_sp_factor (Special Region Factor)',
        zaxis=dict(
            title='Relative Loss',
            range=z_range  # Fixed z-axis range
        ),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.3)
        ),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.7)
    ),
    width=1000,
    height=800,
    font=dict(size=12),
    showlegend=True,
    legend=dict(x=0.02, y=0.98, xanchor='left', yanchor='top')
)

fig_3d.show()


In [15]:
# Calculate global y-axis range for consistent scaling across all 2D plots
y_min = np.nanmin(relative_loss)
y_max = np.nanmax(relative_loss)
y_range = [y_min * 0.95, y_max * 1.05]  # Add 5% padding

# Create frames for horizontal cross-sections (varying alpha_sp_factor)
frames_horiz = []
slider_steps_horiz = []

sampled_indices_horiz = list(range(0, len(alpha_bg_values), max(1, len(alpha_bg_values)//50)))

for idx, i in enumerate(sampled_indices_horiz):
    frame_data = [
        go.Scatter(
            x=alpha_sp_factors,
            y=relative_loss[i, :],
            mode='lines+markers',
            line=dict(color='red', width=2),
            marker=dict(size=5),
            name=f'alpha_bg = {alpha_bg_values[i]:.2f}'
        ),
        go.Scatter(
            x=[alpha_sp_factors[min_idx[1]]],
            y=[relative_loss[i, min_idx[1]]],
            mode='markers',
            marker=dict(size=12, color='lime', symbol='star', 
                       line=dict(width=2, color='black')),
            name='Optimal alpha_sp_factor',
            showlegend=False
        )
    ]
    
    frames_horiz.append(go.Frame(data=frame_data, name=str(idx)))
    
    slider_steps_horiz.append({
        "args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
        "label": f"{alpha_bg_values[i]:.0f}",
        "method": "animate"
    })

# Create figure for horizontal cross-sections
fig_horiz = go.Figure(
    data=frames_horiz[0].data,
    frames=frames_horiz
)

# Update layout
fig_horiz.update_layout(
    title=dict(
        text='Cross-section: Fixed alpha_bg, Varying alpha_sp_factor',
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    xaxis=dict(
        title='alpha_sp_factor (Special Region Factor)'
    ),
    yaxis=dict(
        title='Relative Loss',
        range=y_range
    ),
    width=1000,
    height=500,
    showlegend=False,
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "y": -0.15,
        "xanchor": "left",
        "currentvalue": {
            "prefix": "alpha_bg = ",
            "visible": True,
            "xanchor": "left"
        },
        "transition": {"duration": 0},
        "pad": {"b": 10, "t": 10},
        "len": 0.9,
        "x": 0.05,
        "steps": slider_steps_horiz
    }]
)

fig_horiz.show()


In [16]:
# Create frames for vertical cross-sections (varying alpha_bg)
frames_vert = []
slider_steps_vert = []

sampled_indices_vert = list(range(0, len(alpha_sp_factors), max(1, len(alpha_sp_factors)//50)))

for idx, j in enumerate(sampled_indices_vert):
    frame_data = [
        go.Scatter(
            x=alpha_bg_values,
            y=relative_loss[:, j],
            mode='lines+markers',
            line=dict(color='blue', width=2),
            marker=dict(size=5),
            name=f'alpha_sp_factor = {alpha_sp_factors[j]:.2f}'
        ),
        go.Scatter(
            x=[alpha_bg_values[min_idx[0]]],
            y=[relative_loss[min_idx[0], j]],
            mode='markers',
            marker=dict(size=12, color='lime', symbol='star', 
                       line=dict(width=2, color='black')),
            name='Optimal alpha_bg',
            showlegend=False
        )
    ]
    
    frames_vert.append(go.Frame(data=frame_data, name=str(idx)))
    
    slider_steps_vert.append({
        "args": [[str(idx)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
        "label": f"{alpha_sp_factors[j]:.2f}",
        "method": "animate"
    })

# Create figure for vertical cross-sections
fig_vert = go.Figure(
    data=frames_vert[0].data,
    frames=frames_vert
)

# Update layout
fig_vert.update_layout(
    title=dict(
        text='Cross-section: Fixed alpha_sp_factor, Varying alpha_bg',
        x=0.5,
        xanchor='center',
        font=dict(size=16)
    ),
    xaxis=dict(
        title='alpha_bg (Background Stiffness)'
    ),
    yaxis=dict(
        title='Relative Loss',
        range=y_range
    ),
    width=1000,
    height=500,
    showlegend=False,
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "y": -0.15,
        "xanchor": "left",
        "currentvalue": {
            "prefix": "alpha_sp_factor = ",
            "visible": True,
            "xanchor": "left"
        },
        "transition": {"duration": 0},
        "pad": {"b": 10, "t": 10},
        "len": 0.9,
        "x": 0.05,
        "steps": slider_steps_vert
    }]
)

fig_vert.show()


In [ ]:
data = np.load('cone_verification_deformation.npz', allow_pickle=True)

print('='*80)
print('EXACT VALUES IN PREPROCESSED DATA')
print('='*80)
print()

# Get alpha, beta, kappa values
alpha_k = data['alpha_k']
beta_k = data['beta_k']
kappa_k = data['kappa_k']
labels = data['labels']

print('Ground Truth Material Parameters (from preprocessed file):')
print('-'*80)

# Background region (label 0)
bg_mask = labels == 0
print(f'Background region (label=0): {bg_mask.sum()} elements')
print(f'  alpha = {alpha_k[bg_mask][0]:.10f}')
print(f'  beta = {beta_k[bg_mask][0]:.10f}')
print(f'  kappa = {kappa_k[bg_mask][0]:.10f}')
print()

# Special region (label 1)
sp_mask = labels == 1
print(f'Special region (label=1): {sp_mask.sum()} elements')
print(f'  alpha = {alpha_k[sp_mask][0]:.10f}')
print(f'  beta = {beta_k[sp_mask][0]:.10f}')
print(f'  kappa = {kappa_k[sp_mask][0]:.10f}')
print()

In [ ]:
alpha_k[sp_mask]